In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#hide
! [ -e /content ] && pip install -Uqq fastbook

In [3]:
pip install fastbook --upgrade

In [5]:
import fastbook

In [6]:
fastbook.setup_book()
from fastbook import *

In [7]:
from fastcore.all import *
import time

def search_images(term, max_images=200):
    url = 'https://duckduckgo.com/'
    res = urlread(url,data={'q':term})
    searchObj = re.search(r'vqd=([\d-]+)\&', res)
    requestUrl = url + 'i.js'
    params = dict(l='us-en', o='json', q=term, vqd=searchObj.group(1), f=',,,', p='1', v7exp='a')
    urls,data = set(),{'next':1}
    while len(urls)<max_images and 'next' in data:
        data = urljson(requestUrl,data=params)
        urls.update(L(data['results']).itemgot('image'))
        requestUrl = url + data['next']
        time.sleep(0.2)
    return L(urls)[:max_images]

In [9]:
from fastdownload import download_url
from fastai.vision.all import *

In [20]:
searches = 'photo of burger','photo of pizza'
downloaded_path = Path("downloaded_food")
resized_path = Path("resized_food")

for o in searches:
    downloaded_dest = (downloaded_path/o)
    downloaded_dest.mkdir(exist_ok=True, parents=True)
    download_images(downloaded_dest, urls=search_images(f'{o} photo'))
    resize_images(downloaded_path/o, max_size=400, dest=resized_path/o)

In [21]:
failed = verify_images(get_image_files(resized_path))
failed.map(Path.unlink)
len(failed)

In [22]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(resized_path)

dls.show_batch(max_n=20)

In [23]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(30)

In [40]:
uploader = SimpleNamespace(data = ['../input/testdata/Food-5.jpeg'])

In [34]:
download_url(search_images('photo of a burger', max_images=1)[0], 'burger.jpg', show_progress=False)
Image.open('burger.jpg').to_thumb(256,256)

In [41]:
is_burger,_,probs = learn.predict(PILImage.create(uploader.data[0]))
print(f"This is a: {is_burger}.")
print(f"Probability it's a burger: {probs[0]:.4f}")